In [2]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from ast import literal_eval
import re
import os
import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')


# Forecasts Data

In [3]:
forecastsFilename = '../../data/sentences-20200407.xlsx'
forecastsData = pd.read_excel(forecastsFilename)

In [15]:
forecastsData['source']

0           PCR
1           PCR
2           PCR
3           PCR
4           PCR
          ...  
55363    tagged
55364    tagged
55365    tagged
55366    tagged
55367    tagged
Name: source, Length: 55368, dtype: object

In [4]:
forecastsData['annotatedBy'] = forecastsData['annotatedBy'].fillna("")
forecastsData['annotationTitle'] = forecastsData['annotationTitle'].fillna("")
forecastsData['machine generated'] = forecastsData['machine generated'].fillna("")
forecastsData['annotationSummary'] = forecastsData['annotationSummary'].fillna("")
forecastsData['lastAnnotated'] = forecastsData['lastAnnotated'].fillna("")
forecastsData['Lesson Strength'] = forecastsData['Lesson Strength'].fillna(-1)
forecastsData['topic'] = forecastsData['topic'].fillna(-1)
forecastsData['topTopics'] = forecastsData['topTopics'].fillna("[]")
forecastsData['related_lessons'] = forecastsData['related_lessons'].fillna("[]")

In [75]:
related_lessons = forecastsData['related_lessons']

In [184]:
updated_related_lessons = []
for lessons in related_lessons:
    if lessons == lessons:
        updated_related_lessons.append(lessons.replace('  ', ' ').replace('[ ', '[').replace(' ]', ']').replace(' ', ', '))
#         print(lessons.replace(' ', ', '))
    else:
        updated_related_lessons.append('[]')
forecastsData['related_lessons'] = updated_related_lessons

In [6]:
forecastsData['related_lessons'].unique()

array(['[7, 3187, 2882, 5, 21956]', '[39768, 6, 13695, 13691, 17120]',
       '[41, 39760, 6352, 7839, 3392]', ...,
       '[57958, 2771, 464, 2777, 47989]',
       '[29235, 5125, 1622, 14006, 1158]',
       '[4111, 2776, 2775, 34006, 34003]'], dtype=object)

In [5]:
# Converting dataframe to elasticsearch format
folderName = '../../data/'
i = 1
for index, row in forecastsData.iterrows():
    row['paragraph'] = re.sub(r'\d{1,3}\.\s', '', str(row['paragraph']).replace('\n', ' ').replace('"', "'"))
    row['Sectors'] = str(row['Sectors']).replace("'", '"')
    f = None
    if(i > 0 and i <= 20000):
        f = open(folderName + "sentences." + str(1) + ".ndjson", "a+")
    elif(i > 20000 and i <= 40000):
        f = open(folderName + "sentences." + str(2) + ".ndjson", "a+")
    elif(i > 40000 and i <= 60000):
        f = open(folderName + "sentences." + str(3) + ".ndjson", "a+")
    elif(i > 60000 and i <= 80000):
        f = open(folderName + "sentences." + str(4) + ".ndjson", "a+")
    elif(i > 80000 and i <= 100000):
        f = open(folderName + "sentences." + str(5) + ".ndjson", "a+")
    allString = '{"index":{"_id": "' + str(row['id']) +'"}}\n'
    allString = allString + '{ \
        "referenceId": "'+ str(row['reference_id']).zfill(20) + '", \
        "title": "'+ str(row['Title']).replace('\n', ' ').replace('"', "'") + '", \
        "paragraph": "'+ str(row['paragraph']) + '", \
        "themes": '+ str(row['Themes']) + ', \
        "sectors": '+ str(row['Sectors']) + ', \
        "countries": '+ str(row['Countries']).replace('People"', "People'") + ', \
        "lessonStrength": '+ str(row['Lesson Strength']) + ', \
        "relatedLessons": '+ str(row['related_lessons']) + ', \
        "topic": '+ str(row['topic']) + ', \
        "topTopics": '+ str(row['topTopics']) + ', \
        "modalities": '+ str(row['Modalities']).replace("'", '"') + ', \
        "lessonType": "Self Evaluation", \
        "isLesson": ' + str(row['isLesson']).lower() + ',  \
        "month": "' + str(row['Month']) + '",  \
        "year": ' + str(row['Year']) + ',  \
        "annotationTitle": "' + str(row['annotationTitle']) + '", \
        "summary": "' + str(row['machine generated']).replace('\n', ' ').replace('"', "'").title() + '", \
        "context": "",  \
        "annotatedBy": "' + str(row['annotatedBy']) + '", \
        "annotationSummary": "' + str(row['annotationSummary']) + '", \
        "lastAnnotated": "' + str(row['lastAnnotated']) + '", \
        "annotationStatus": "' + str(row['annotationStatus']) + '", \
        "urlToFile": "' + str(row['Download Link']) + '", \
        "lastUpdated": "",  \
        "source": "' + str(forecastsData.at[index, 'source']) + '" \
    }\n'
    i = i + 1
    f.write(allString)
    f.close()
#     if(i==10000):
#         break

# Topics Data

In [3]:
topicsFileName = '../../data/topics_20200325.xlsx'
topics = pd.read_excel(topicsFileName)

In [4]:
topics

,Unnamed: 0,Topic Number,Keywords,Keywords Frequencies,Number of Lessons,Number of PCRs,Keywords Weights,Top Word,x,y,adjacentTopics
0,0,0,"['statement', 'audit', 'financial', 'accountin...","[194, 426, 2052, 138, 47, 55, 47, 246, 26, 19,...",9857,957,"[127, 115, 57, 32, 25, 20, 19, 12, 11, 10, 9, ...",statement,0.133656,-0.000225,"[1, 2, 4, 8, 12]"
1,1,1,"['maintenance', 'road', 'covenant', 'budget', ...","[1254, 2037, 1045, 915, 633, 5580, 1517, 1239,...",10567,963,"[327, 282, 231, 170, 130, 129, 128, 107, 107, ...",maintenance,0.038072,0.025617,"[0, 4, 5, 8, 12]"
2,2,2,"['website', 'grievance', 'complaint', 'upload'...","[69, 30, 25, 9, 19, 10, 175, 4, 23, 29, 7, 3, ...",9798,957,"[58, 17, 14, 12, 9, 6, 3, 3, 2, 2, 2, 2, 2, 2,...",website,0.136032,-0.003685,"[0, 4, 8, 12]"
3,3,3,"['loan', 'delay', 'adb', 'implementation', 'co...","[2887, 1455, 4184, 4980, 1120, 1614, 675, 525,...",10700,961,"[331, 197, 159, 150, 116, 115, 89, 86, 85, 79,...",loan,-0.043908,0.167825,[]
4,4,4,"['road', 'construction', 'rehabilitation', 'sa...","[2037, 1011, 291, 338, 243, 169, 1231, 274, 17...",10326,960,"[295, 135, 88, 80, 69, 66, 63, 63, 53, 52, 52,...",road,0.093660,-0.004680,"[0, 1, 2, 8, 9, 12]"
5,5,5,"['financial', 'cost', 'credit', 'bank', 'marke...","[2536, 1688, 840, 876, 1295, 2856, 861, 612, 1...",10673,963,"[218, 214, 164, 159, 151, 122, 100, 91, 81, 78...",financial,-0.009646,-0.018591,"[1, 7, 9]"
6,6,6,"['implementation', 'agency', 'consultant', 'st...","[5028, 2527, 1479, 1735, 1878, 2955, 1529, 244...",10743,964,"[307, 211, 158, 154, 154, 127, 125, 117, 117, ...",implementation,-0.148977,0.128479,[]
7,7,7,"['government', 'law', 'regulation', 'establish...","[5794, 505, 481, 1256, 459, 667, 1544, 1592, 4...",10695,963,"[215, 92, 74, 74, 71, 67, 66, 64, 62, 60, 59, ...",government,-0.093109,-0.009298,[5]
8,8,8,"['performance', 'report', 'evaluation', 'impac...","[2408, 1684, 1336, 1279, 1301, 1963, 526, 900,...",10460,963,"[702, 516, 381, 202, 156, 154, 148, 127, 116, ...",performance,0.114196,0.049000,"[0, 1, 2, 4, 12]"
9,9,9,"['water', 'system', 'power', 'supply', 'tariff...","[1064, 2207, 660, 683, 467, 313, 997, 2324, 11...",10530,963,"[216, 146, 140, 133, 95, 81, 78, 78, 69, 64, 6...",water,0.035703,-0.074938,"[4, 5]"


In [6]:
i = 0
for index, row in topics.iterrows():
    f = open("../../data/topics.ndjson", "a+")
#     allString = '{"index":{"_id": "' + str(row['Topic Number']) +'"}}\n'
    allString = '{ "index" : { "_index" : "topics", "_type" : "type1" } }\n'
    allString = allString + '{ \
        "key": '+ str(topics.at[index, 'Topic Number']) + ', \
        "numberOfLessons": '+  str(topics.at[index, 'Number of Lessons']) + ', \
        "numberOfPCRs": '+  str(topics.at[index, 'Number of PCRs']) + ', \
        "x": '+  str(topics.at[index, 'x']) + ', \
        "y": '+  str(topics.at[index, 'y']) + ', \
        "topWord": "'+ str(topics.at[index, 'Top Word']) + '", \
        "adjacentTopics": '+ str(topics.at[index, 'adjacentTopics']) + ', \
        "keywords": '+ str(topics.at[index, 'Keywords']).replace("'", '"') +', \
        "frequencies": '+ str(topics.at[index, 'Keywords Weights']) + ' \
    }\n'
    i = i + 1
    f.write(allString)
    f.close()

# PCRs Data

In [ ]:
PCRsFileName = '../data/pcrs_2020-02-25.xlsx'
PCRs = pd.read_excel(topicsFileName)

In [ ]:
# Converting dataframe to elasticsearch format
i = 1
for index, row in PCRs.iterrows():
    row['paragraph'] = re.sub(r'\d{1,3}\.\s', '', str(row['paragraph']).replace('\n', ' ').replace('"', "'"))
    row['sectors'] = str(row['sectors']).replace('"', "'")
    f = None
    if(i > 0 and i <= 20000):
        f = open(folderName + "pcrs." + str(1) + ".elastic", "a+")
    elif(i > 20000 and i <= 40000):
        f = open(folderName + "pcrs." + str(2) + ".elastic", "a+")
    allString = '{"index":{"_id": "' + str(row['id']) +'"}}\n'
    allString = allString + '{ \
        "referenceId": "'+ str(row['reference_id']).zfill(20) + '", \
        "title": "'+ str(row['Title']).replace('\n', ' ').replace('"', "'") + '", \
        "paragraph": "'+ str(row['paragraph']) + '", \
        "themes": '+ str(row['Themes']) + ', \
        "sectors": '+ str(row['Sectors']) + ', \
        "countries": '+ str(row['Countries']).replace('People"', "People'") + ', \
        "lessonStrength": '+ str(row['Lesson Strength']) + ', \
        "relatedLessons": '+ str(row['related_lessons']) + ', \
        "topic": '+ str(row['topic']) + ', \
        "topTopics": '+ str(row['topTopics']) + ', \
        "modalities": '+ str(row['Modalities']).replace("'", '"') + ', \
        "lessonType": "Self Evaluation", \
        "isLesson": ' + str(row['isLesson']).lower() + ',  \
        "month": "' + str(row['Month']) + '",  \
        "year": ' + str(row['Year']) + ',  \
        "annotationTitle": "' + str(row['annotationTitle']) + '", \
        "summary": "' + str(row['machine generated']).replace('\n', ' ').replace('"', "'").title() + '", \
        "context": "",  \
        "annotatedBy": "' + str(row['annotatedBy']) + '", \
        "annotationSummary": "' + str(row['annotationSummary']) + '", \
        "lastAnnotated": "' + str(row['lastAnnotated']) + '", \
        "annotationStatus": "' + str(row['annotationStatus']) + '", \
        "urlToFile": "' + str(row['Download Link']) + '", \
        "lastUpdated": "",  \
        "source": "PCR" \
    }\n'
    i = i + 1
    f.write(allString)
    f.close()
#     if(i==10000):
#         break